In [2]:
import datetime

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import datetime
from matplotlib.dates import DateFormatter

In [96]:
data_dt = pd.read_csv("only_dt.csv")
data_esf = pd.read_csv("esf_all.csv")
data_ch = pd.read_csv("only_checks.csv")

data_dt.head(19)

C:\Users\DIE4RMK\AppData\Local\Temp\ipykernel_13640\2792495138.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data_esf = pd.read_csv("esf_all.csv")


,dt_date,nom_razr,g32goodsnumeric,g34origincountryname,g38netweightquantity,g14rbunp,g08rbunp,g331goodstnvedcode,"ТС/кг, бел руб",ГОД,МЕСЯЦ
0,2022-03-15,06649/22004631,1,КОСТА-РИКА,180972,799797409,799797409,803901000,29.43,2022,3
1,2022-03-15,06649/22004636,1,ПАНАМА,180612,799797409,799797409,803901000,29.52,2022,3
2,2022-03-15,06650/22003657,1,ЭКВАДОР,175050,800007064,800007064,803901000,31.77,2022,3
3,2022-03-25,06650/22003852,1,ЭКВАДОР,170316,794878660,794878660,803901000,22.14,2022,3
4,2022-03-28,06649/22005210,1,ЭКВАДОР,185760,799797409,799797409,803901000,25.65,2022,3
5,2022-03-28,06649/22005211,1,ЭКВАДОР,185220,799797409,799797409,803901000,25.65,2022,3
6,2022-03-15,06650/22003645,1,ЭКВАДОР,178812,707444748,707444748,803901000,29.43,2022,3
7,2022-03-15,06533/22009550,1,ПАНАМА,171873,707768747,707768747,803901000,28.44,2022,3
8,2022-03-25,06533/22010317,1,ПАНАМА,178083,797778804,797778804,803901000,27.00,2022,3
9,2022-04-04,06649/22005588,1,ЭКВАДОР,184320,799797409,799797409,803901000,21.24,2022,4


In [90]:

"""
2.8
Подсчет процента присутствия компании в интервале
"""
def count_interval(left, right):
    """
    Функция вычисления дельты - размерности временного интервала
    """
    if isinstance(left, datetime.datetime) & isinstance(right, datetime.datetime): # Проверка на формат datetime
        return int((right - left).days) # Вычисление лямбды
    else:
        return 0 # ERROR
        
    
def get_activity_percent_sells(df):
    interval = count_interval(time_start, time_end)
    """
    Функия расчета отношения "торговых" дней к заданому интервалу для продаж (provider_unp)
    """
    unique_companies = df['provider_unp'].unique() #Получаем список 
    percents = pd.DataFrame(columns=['Seller', 'percent']) # Храним колонку - компания и ее процент
    for company in unique_companies: 
        active_dates = df[df["provider_unp"] == company]["general_date_transaction"].nunique() # Подсчет кол-ва сделок конкретной компании.
        percent = round(active_dates / interval * 100, 1) # Получаем процент и округляем
        new_row = {'Seller': company, 'percent': percent}
        percents.loc[len(percents)] = new_row
    return percents
    
def get_activity_percent_buyers(df):
        interval = count_interval(time_start, time_end)
        """
        Функия расчета отношения "торговых" дней к заданому интервалу для покупок (recipient_unp)
        """
        unique_companies = df['recipient_unp'].unique() #Получаем список 
        percents = pd.DataFrame(columns=['Buyer', 'percent']) # Храним колонку - компания и ее процент
        for company in unique_companies: 
            active_dates = df[df["recipient_unp"] == company]["general_date_transaction"].nunique() # Подсчет кол-ва сделок конкретной компании.
            percent = round(active_dates / interval * 100, 1) # Получаем процент и округляем
            new_row = {'Buyer': company, 'percent': percent}
            percents.loc[len(percents)] = new_row
        return percents

In [94]:
"""
БЛОК ТЕСТ
"""




time_start = "2022-01-01"
time_start = pd.to_datetime(time_start)

time_end = "2022-02-01"
time_end = pd.to_datetime(time_end)

interval = count_interval(time_start, time_end)

ans = get_activity_percent_sells(data_esf)

print(ans)

      Seller  percent
0  707484767     19.4
1  707777798     19.4
2  797778804      3.2
3  700940649      3.2
4  800847488      3.2
5  697846977      3.2
6  900794744      6.5
7  700997964      9.7
8        NaN      0.0
9  490490778     12.9
